In [24]:
import numpy as np
import pandas as pd
from missingpy import MissForest
from sklearn.impute import KNNImputer, SimpleImputer

# Reading Dataset

In [25]:
# Read the dataset
raw = pd.read_csv("data/nyc_taxi.csv",low_memory=False)
raw['timestamp'] = pd.to_datetime(raw['timestamp'])

# Preview raw dataset
raw

,timestamp,value
0,2014-07-01 00:00:00,10844
1,2014-07-01 00:30:00,8127
2,2014-07-01 01:00:00,6210
3,2014-07-01 01:30:00,4656
4,2014-07-01 02:00:00,3820
...,...,...
10315,2015-01-31 21:30:00,24670
10316,2015-01-31 22:00:00,25721
10317,2015-01-31 22:30:00,27309
10318,2015-01-31 23:00:00,26591


In [26]:
# The times of anomaly events (Ground Truth)
anomaly_points = [
        [
            "2014-10-30 15:30:00.000000",
            "2014-11-03 22:30:00.000000"
        ],
        [
            "2014-11-25 12:00:00.000000",
            "2014-11-29 19:00:00.000000"
        ],
        [
            "2014-12-23 11:30:00.000000",
            "2014-12-27 18:30:00.000000"
        ],
        [
            "2014-12-29 21:30:00.000000",
            "2015-01-03 04:30:00.000000"
        ],
        [
            "2015-01-24 20:30:00.000000",
            "2015-01-29 03:30:00.000000"
        ]
]

# Labeling: if anomaly then 1 else 0
raw['anomaly'] = 0  # Set default values
for start, end in anomaly_points:
    raw.loc[((raw['timestamp'] >= start) & (raw['timestamp'] <= end)), 'anomaly'] = 1

# Preview labeled raw dataset
raw

,timestamp,value,anomaly
0,2014-07-01 00:00:00,10844,0
1,2014-07-01 00:30:00,8127,0
2,2014-07-01 01:00:00,6210,0
3,2014-07-01 01:30:00,4656,0
4,2014-07-01 02:00:00,3820,0
...,...,...,...
10315,2015-01-31 21:30:00,24670,0
10316,2015-01-31 22:00:00,25721,0
10317,2015-01-31 22:30:00,27309,0
10318,2015-01-31 23:00:00,26591,0


In [27]:
raw['anomaly'].value_counts()

0    9285
1    1035
Name: anomaly, dtype: int64

# Preprocessing

In [28]:
# Convert the timestamp
df = pd.DataFrame()
df['year'] = raw['timestamp'].dt.year
df['month'] = raw['timestamp'].dt.month
df['day'] = raw['timestamp'].dt.day
df['hour'] = raw['timestamp'].dt.hour
df['value'] = raw['value']
df['anomaly'] = raw['anomaly']

# delete unused dataframe
del raw

# Preview dataset
df

,year,month,day,hour,value,anomaly
0,2014,7,1,0,10844,0
1,2014,7,1,0,8127,0
2,2014,7,1,1,6210,0
3,2014,7,1,1,4656,0
4,2014,7,1,2,3820,0
...,...,...,...,...,...,...
10315,2015,1,31,21,24670,0
10316,2015,1,31,22,25721,0
10317,2015,1,31,22,27309,0
10318,2015,1,31,23,26591,0


In [29]:
# Calculate the number of rows representing 80% of the DataFrame for training
num_rows = int(0.8 * len(df))

# Get the first 80% of the DataFrame
df_train = df[:num_rows]

# Get the remaining 20% of the DataFrame
df_test = df[num_rows:]

# delete unused dataframe
del df

In [30]:
df_train['anomaly'].value_counts()

0    7842
1     414
Name: anomaly, dtype: int64

In [31]:
df_test['anomaly'].value_counts()

0    1443
1     621
Name: anomaly, dtype: int64

In [32]:
# (1) Get normal training data, (2) Drop 'anomaly' column, and (3) convert it to float
X_train_normal = df_train[df_train['anomaly'] == 0].drop(columns=['anomaly']).to_numpy(dtype=float)
# Copy the 'value' column of normal training data
y_train_normal = X_train_normal[:, -1].copy()

# (1) Get anomaly training data, (2) Drop 'anomaly' column, and (3) convert it to float
X_train_anomaly = df_train[df_train['anomaly'] == 1].drop(columns=['anomaly']).to_numpy(dtype=float)
# Copy the 'value' column of normal anomaly data
y_train_anomaly = X_train_anomaly[:, -1].copy()


In [33]:
# (1) Get normal testing data, (2) Drop 'anomaly' column, and (3) convert it to float
X_test_normal = df_test[df_test['anomaly'] == 0].drop(columns=['anomaly']).to_numpy(dtype=float)
# Copy the 'value' column of normal testing data
y_test_normal = X_test_normal[:, -1].copy()

# (1) Get anomaly testing data, (2) Drop 'anomaly' column, and (3) convert it to float
X_test_anomaly = df_test[df_test['anomaly'] == 1].drop(columns=['anomaly']).to_numpy(dtype=float)
# Copy the 'value' column of normal anomaly data
y_test_anomaly = X_test_anomaly[:, -1].copy()


In [34]:
X_train_normal.shape

(7842, 5)

In [35]:
X_train_anomaly.shape

(414, 5)

In [36]:
X_test_normal.shape

(1443, 5)

In [37]:
y_test_anomaly.shape

(621,)

In [38]:
df_train[df_train['anomaly'] == 0].drop(columns=['anomaly'])

,year,month,day,hour,value
0,2014,7,1,0,10844
1,2014,7,1,0,8127
2,2014,7,1,1,6210
3,2014,7,1,1,4656
4,2014,7,1,2,3820
...,...,...,...,...,...
8251,2014,12,19,21,26403
8252,2014,12,19,22,26905
8253,2014,12,19,22,26723
8254,2014,12,19,23,25807


In [39]:
X_train_normal

array([[2.0140e+03, 7.0000e+00, 1.0000e+00, 0.0000e+00, 1.0844e+04],
       [2.0140e+03, 7.0000e+00, 1.0000e+00, 0.0000e+00, 8.1270e+03],
       [2.0140e+03, 7.0000e+00, 1.0000e+00, 1.0000e+00, 6.2100e+03],
       ...,
       [2.0140e+03, 1.2000e+01, 1.9000e+01, 2.2000e+01, 2.6723e+04],
       [2.0140e+03, 1.2000e+01, 1.9000e+01, 2.3000e+01, 2.5807e+04],
       [2.0140e+03, 1.2000e+01, 1.9000e+01, 2.3000e+01, 2.6432e+04]])

In [40]:
y_train_normal

array([10844.,  8127.,  6210., ..., 26723., 25807., 26432.])

# Metrix Calculation

In [41]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler


def RMSE(original, filled):
    score = np.sqrt(mean_squared_error(original, filled))
    return score


def MAE(original, filled):
    score = mean_absolute_error(original, filled)
    return score


def MAPE(original, filled):
    score = mean_absolute_percentage_error(original, filled)
    return score


def metric_calc(original, filled):
    y_true = original.reshape(-1, 1)
    y = filled.reshape(-1, 1)

    scaler = MinMaxScaler()
    y_true_scaled = scaler.fit_transform(y_true)
    y_scaled = scaler.fit_transform(y)

    rmse = RMSE(y_true_scaled, y_scaled)
    print("RMSE=", rmse)

    mae = MAE(y_true_scaled, y_scaled)
    print("MAE=", mae)

    mape = MAPE(y_true_scaled, y_scaled)
    print("MAPE=", mape)

# Finding Threshold

In [42]:
def calc_threshold(imputer, X, y, imputed_col=-1):
    # Copy the dataset before removing some values
    X_nan = np.copy(X)

    # Calculate the number of rows to select (10% of the total rows)
    num_rows_to_select = int(0.1 * X_nan.shape[0])

    # Randomly select row indices to change
    random_indices = np.random.choice(X_nan.shape[0], size=num_rows_to_select, replace=False)

    # Change the value column to NaN for the selected rows
    X_nan[random_indices, imputed_col] = np.nan

    X_imputation = imputer.fit_transform(X_nan)

    metric_calc(y[random_indices], X_imputation[random_indices, imputed_col])

Normal data

In [43]:
imputer = KNNImputer(n_neighbors=4, weights="uniform")
calc_threshold(imputer, X_train_normal, y_train_normal)

RMSE= 0.10523792191152789
MAE= 0.07766683367177697
MAPE= 62621187160.206566


In [44]:
imputer = KNNImputer(n_neighbors=4, weights="uniform")
calc_threshold(imputer, X_train_anomaly, y_train_anomaly)

RMSE= 0.10498989181476096
MAE= 0.07156172917660168
MAPE= 0.16082960909292474


In [45]:
imputer = KNNImputer(n_neighbors=4, weights="uniform")
calc_threshold(imputer, X_test_normal, y_test_normal)

RMSE= 0.09317761917091626
MAE= 0.0647567524329333
MAPE= 0.3977597955614908


In [46]:
imputer = KNNImputer(n_neighbors=4, weights="uniform")
calc_threshold(imputer, X_test_anomaly, y_test_anomaly)

RMSE= 0.10808187722814311
MAE= 0.07771261193736982
MAPE= 0.2907825308376493


In [47]:
# Calculate the number of rows to select (10% of the total rows)
num_rows_to_select = int(0.1 * X_train_normal.shape[0])

# Randomly select row indices to change
random_indices = np.random.choice(X_train_normal.shape[0], size=num_rows_to_select, replace=False)

# Change the value column to NaN for the selected rows
X_train_normal[random_indices, -1] = np.nan

In [48]:
imputer = KNNImputer(n_neighbors=4, weights="uniform")
X_train_imputation = imputer.fit_transform(X_train_normal)

In [49]:
metric_calc(y_train_normal[random_indices], X_train_imputation[random_indices, -1])

RMSE= 0.10828293487949135
MAE= 0.08343701398638009
MAPE= 104303084688.94331


Anomaly data

In [50]:
# Calculate the number of rows to select (10% of the total rows)
num_rows_to_select = int(0.1 * X_train_anomaly.shape[0])

# Randomly select row indices to change
random_indices = np.random.choice(X_train_anomaly.shape[0], size=num_rows_to_select, replace=False)

# Change the value column to NaN for the selected rows
X_train_anomaly[random_indices, -1] = np.nan

In [51]:
imputer = KNNImputer(n_neighbors=4, weights="uniform")
X_train_imputation = imputer.fit_transform(X_train_anomaly)

In [52]:
X_train_anomaly[random_indices].shape

(41, 5)

In [53]:
X_train_imputation[random_indices, -1].shape

(41,)

In [54]:
metric_calc(y_train_anomaly[random_indices], X_train_imputation[random_indices, -1])

RMSE= 0.1108337396993559
MAE= 0.08400304599836739
MAPE= 1996924814148.8506
